In [1]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import matplotlib
import pandas as pd
import seaborn as sns
from scipy.stats import gaussian_kde

plt.rc('font', family='serif')
plt.rc('text.latex', preamble=r'\usepackage{amsmath,bm}')
matplotlib.rcParams.update({'font.size': 16})

# matplotlib.use('TkAgg')

In [74]:
infile = "data_test_boozer.pickle"
indata = pickle.load(open(infile,'rb'))
c_times = indata['c_times']
mu_crit =indata['mu_crit']
modB = indata['modB']
stz_inits = indata['stz_inits'] 
vpar_inits = indata['vpar_inits']
s_pdf = indata['s_pdf']
s_mean = indata['s_mean']
s_std = indata['s_std']
mu = indata['mu']
mu_mean = indata['mu_mean']
mu_std = indata['mu_std']


KeyError: 's_pdf'

In [75]:
energy_func = lambda t:  3.5*np.exp(-2*t/tmax)
energy = energy_func(c_times)

# 1. Control Variate

## 1.1 Control variate using s

In [73]:
s = stz_inits[:,0]
s_mean = 0.20596158592366795
s_var =  0.025053175240927983
s_std = np.sqrt(var_s)
rho = np.corrcoef(s,c_times)[0,1]
c = -rho* np.std(c_times)/s_std
# control variate
cv = c_times + c*(s-s_mean)
print('MC mean',np.mean(c_times))
print('MC std',np.std(c_times))
print('CV mean',np.mean(cv))
print('CV std',np.std(cv))
print("Reduction in standard deviation",1-np.std(cv)/np.std(c_times))
alpha = np.std(cv)/np.std(c_times)
print("Effective increase in number of samples",1/alpha**2)

MC mean 0.007081968741085875
MC std 0.004490661214463294
CV mean 0.007122153702638998
CV std 0.004207876413584632
Reduction in standard deviation 0.06297175123518184
Effective increase in number of samples 1.1389237052071695


## 1.2 Control variate using mu

In [72]:
mu_mean = 9139072590066.031 
mu_std = 4121149626307.4717
rho = np.corrcoef(mu,c_times)[0,1]
c_times_std = np.std(c_times)
c = -rho*c_times_std/mu_std
# control variate
cv = c_times + c*(mu-mu_mean)
print('MC mean',np.mean(c_times))
print('MC std',np.std(c_times))
print('CV mean',np.mean(cv))
print('CV std',np.std(cv))
print("Reduction in standard deviation",1-np.std(cv)/np.std(c_times))
alpha = np.std(cv)/np.std(c_times)
print("Effective increase in number of samples",1/alpha**2)

MC mean 0.007081968741085875
MC std 0.004490661214463294
CV mean 0.007096295667732925
CV std 0.003934286015910921
Reduction in standard deviation 0.12389605271500503
Effective increase in number of samples 1.3028329263181768


## 1.3 Control Variate using s and mu

In [71]:
"""
Use both s and mu as control variates. In this formulation we take a shortcut to building the control variate. 
We first find the optimal estimator when mu is a control variate,
    cv1 = Y + c1*(mu - mean(mu))
    c1 = -corr(mu,y)*np.std(c_times)/std(mu)
Then, with c1 fixed, we use S as a control variate with cv1. i.e. we use the control variate
    cv2 = cv1 + c2*(s - mean(s))
    c2 = -corr(s,cv1)*np.std(cv1)/std(s)
"""
rho1 = np.corrcoef(mu,c_times)[0,1]
c1 = -rho1*np.std(c_times)/mu_std
cv1 = c_times + c1*(mu-mu_mean) 
rho2 = np.corrcoef(s,cv1)[0,1]
c2 = -rho2*np.std(cv1)/s_std
cv = cv1 + c2*(s-s_mean)
print('MC mean',np.mean(c_times))
print('MC std',np.std(c_times))
print('CV mean',np.mean(cv))
print('CV std',np.std(cv))
print("Reduction in standard deviation",1-np.std(cv)/np.std(c_times))
alpha = np.std(cv)/np.std(c_times)
print("Effective increase in number of samples",1/alpha**2)

MC mean 0.007081968741085875
MC std 0.004490661214463294
CV mean 0.007134901433288679
CV std 0.003634326055706806
Reduction in standard deviation 0.1906924432416427
Effective increase in number of samples 1.5267671563780767


In [70]:
"""
Use both s and mu as control variates. This is the proper way to construct the control variate.

If Y is the confinement times and X = (mu,s) then we form the control variate
    cv = Y + (X - X_mean) @ c
where c satisfies
    Cov(X,X) @ c = - Cov(Y,X)
Since we know the standard deviations of mu and s, we can input those directly into this equation and simplify a bit. 
Then this equation only relies on correlation coefficients which are scale free. This helps avoid the error that arises
in estimation of coefficients. In short, we find c via the linear system A @ c = - b where
A = [[mu_std, s_std*corr(mu,s)] 
    [mu_std*corr(mu,s), s_std]]
b = y_std *[corr(mu,y),corr(s,y)]
"""
Sigma = np.array([[mu_std,s_std*np.corrcoef(s,mu)[0,1]],[mu_std*np.corrcoef(s,mu)[0,1],s_std]])
rho_s_c = np.corrcoef(s,c_times)[0,1]
rho_mu_c = np.corrcoef(mu,c_times)[0,1]
rhs = np.std(c_times)*np.array([rho_mu_c,rho_s_c])
c = -np.linalg.solve(Sigma,rhs)
X = np.vstack((mu,s)).T
X_mean = np.array([mu_mean,s_mean])
cv = c_times + (X-X_mean) @ c
print('MC mean',np.mean(c_times))
print('MC std',np.std(c_times))
print('CV mean',np.mean(cv))
print('CV std',np.std(cv))
print("Reduction in standard deviation",1-np.std(cv)/np.std(c_times))
alpha = np.std(cv)/np.std(c_times)
print("Effective increase in number of samples",1/alpha**2)

MC mean 0.007081968741085875
MC std 0.004490661214463294
CV mean 0.007134635241801238
CV std 0.003634261210564699
Reduction in standard deviation 0.19070688324034457
Effective increase in number of samples 1.5268216402529167


# Loss Profile

In [6]:
# # loss profile
# fig,ax = plt.subplots(nrows=1,ncols=1,figsize = (10,8))

# # make the loss profile
# times = np.linspace(1e-7,tmax,100)
# loss_profile = np.array([np.mean(c_times< t) for t in times]) 
# plt.plot(times,loss_profile,linewidth=3,label='in-sample')
# loss_profile = np.array([np.mean(c_times< t) for t in times]) 
# plt.plot(times,loss_profile,linewidth=3,label='out-of-sample')

# plt.title("Loss Profile")
# plt.xlabel('time ')
# plt.ylabel("Loss Fraction")
# plt.xscale('log')
# bottom,top =plt.ylim(-0.02,1.02)
# plt.vlines(tmax,bottom,top,linestyle='--',color='k',linewidth=2,label='tmax')
# plt.xlim(1e-7,2*tmax)
# plt.legend()
# plt.show()
